In [ ]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum,lit,when,format_number,row_number,col,filter
from pyspark.sql.types import StringType
from pyspark.sql.window import Window

from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font, Alignment
from openpyxl.worksheet.table import Table, TableStyleInfo

spark = SparkSession.builder \
    .appName("CSV Loader") \
    .config("spark.executor.memory", "32g") \
    .config("spark.driver.memory", "32g") \
    .config("spark.executor.memoryOverhead", "8192") \
    .config("spark.driver.memoryOverhead", "8192") \
    .config("spark.executor.cores", "7") \
    .config("spark.driver.cores", "7") \
    .config("spark.executor.instances", "2") \
    .getOrCreate()


In [ ]:
morefire_file ='/Users/maralsheikhzadeh/documents/codes/Repeating-Analytics/morefire/HG_Salesvalidierung_01-31Mai25.xlsx'
data = []
for sheet_name in ['DE','AT','FR','CH']:
    df = pd.read_excel(morefire_file,header=0,sheet_name=sheet_name)
    df['ID'] = df['ID'].fillna(0).astype(int).astype(str)
    df['LAND'] = sheet_name
    data.append(df)

df_first = pd.concat(data)

## Update Originals first
With Server-connect folder **dbf_to_spark.ipynb**, first update all the csv files below so that we can find the EXT_Verweis in them. The following fields connect the data from all 4 countries into 3 unified tables to further connect them with the file from Morefire.

In [ ]:
V2AD1096 = spark.read.csv('/Volumes/MARAL/CSV/F01/V2AD1096.csv',sep=';',header=True)
V2AD1056 = spark.read.csv('/Volumes/MARAL/CSV/F01/V2AD1056.csv',sep=';',header=True)
V2AD1156 = spark.read.csv('/Volumes/MARAL/CSV/F01/V2AD1156.csv',sep=';',header=True)

In [ ]:
V2AD1096_FR = spark.read.csv('/Volumes/MARAL/CSV/f02/V2AD1096.csv',sep=';',header=True)
V2AD1056_FR = spark.read.csv('/Volumes/MARAL/CSV/f02/V2AD1056.csv',sep=';',header=True)
V2AD1156_FR = spark.read.csv('/Volumes/MARAL/CSV/f02/V2AD1156.csv',sep=';',header=True)

In [ ]:
V2AD1096_AU = spark.read.csv('/Volumes/MARAL/CSV/F03/V2AD1096.csv',sep=';',header=True)
V2AD1056_AU = spark.read.csv('/Volumes/MARAL/CSV/F03/V2AD1056.csv',sep=';',header=True)
V2AD1156_AU = spark.read.csv('/Volumes/MARAL/CSV/F03/V2AD1156.csv',sep=';',header=True)

In [ ]:
V2AD1096_CH = spark.read.csv('/Volumes/MARAL/CSV/F04/V2AD1096.csv',sep=';',header=True)
V2AD1056_CH = spark.read.csv('/Volumes/MARAL/CSV/F04/V2AD1056.csv',sep=';',header=True)
V2AD1156_CH = spark.read.csv('/Volumes/MARAL/CSV/F04/V2AD1156.csv',sep=';',header=True)

In [ ]:
V2AD1096 = V2AD1096_FR.union(V2AD1096_AU).union(V2AD1096_CH).union(V2AD1096)
V2AD1056 = V2AD1056_FR.union(V2AD1056_AU).union(V2AD1056_CH).union(V2AD1056)
V2AD1156 = V2AD1156_FR.union(V2AD1156_AU).union(V2AD1156_CH).union(V2AD1156)


In [ ]:
V2AD1096 = V2AD1096.withColumn('SHOPNUMMER', col('SHOPNUMMER').cast(StringType()))


In [ ]:
df = spark.createDataFrame(df_first)
df = df.join(V2AD1096[['SHOPNUMMER','RECH_NR']],df['ID'] == V2AD1096['SHOPNUMMER'],how='left')

In [ ]:
# merged_df['RECH_NR'] = merged_df['RECH_NR'].astype(str)
df = df.withColumn('RECH_NR', col('RECH_NR').cast(StringType()))
V2AD1056 = V2AD1056.withColumn('RECH_NR', col('RECH_NR').cast(StringType()))

In [ ]:
df = df.join(V2AD1056[['VERWEIS','RECH_NR','BEST_WERT','MWST1','AUF_ANLAGE']],on='RECH_NR',how='left')
df = df.join(V2AD1156[['ART_NR','GROESSE','FARBE','PREIS','MWST','RECHNUNG']],df['RECH_NR']==V2AD1156['RECHNUNG'],how='left')

## Changing from Pyspark to Pandas
I used Pyspark to load and connect the data from our databases because the data is so large and Pandas is so slow. Now that everything is joined and the data is filtered to the original lines we had, it's easier to transform it back to Pandas to do the final analysis on it.

In [ ]:
df = df.toPandas()


In [ ]:
df['GROESSE'] = df['GROESSE'].astype(str).fillna("")
df['FARBE'] = df['FARBE'].astype(str).fillna("")
df['ANR'] = df['ART_NR'].str.ljust(8) + df['GROESSE'].str.ljust(4) + df['FARBE'].str.ljust(2)
df['ANR'] = df['ANR'].str.strip()

In [ ]:
df_new = df.copy()


### Transforming data

- We create a columnn for items that were **Not Found** in the VS4 Data. We make a * sign for them to know what is missing from data.
- The third line removes items that were not computed in Morefire's Sales_Betrag, namely: Versandkosten (VK) and Geschenkverpackung (011P00). Items may need to be added to this list in the future depending on the items that we see further that are not computed here.
- Then we filter only the columns we need. and change data types for our requirements.
- Finally, we aggregate all the lines from the same Rechnung and Extra so that we compute the Fakturierte Nettoumsatz based on the Nettoumsatz value for each product in the list.

In [ ]:
df_new['Publisher_ID'] = df_new['Publisher_ID'].astype(str)
df_new['Not Found'] = ""
df_new.loc[df_new['SHOPNUMMER'].isna(),'Not Found'] = '*'

df_new = df_new[df_new['ART_NR'].isin(['VK','011P00'])==False]
# df_new = df_new[['ID', 'AWIN ID', 'Publisher_ID', 'Sale_Betrag', 'Provision', 'Datum',
#        'Site_Name', 'URL', 'Ablehnungs_Grund', 'Referrer', 'Publisher_URL',
#        'Transaktions_Teile', 'Land_nach_IP', 'Benutzerdef_Parameter',
#        'Produkte', 'getrackte_Teile', 'Netzwerkgebühr','Not Found']]
df_new['PREIS'] = df_new['PREIS'].astype(float).fillna(0)
df_new = df_new.drop_duplicates(subset=['ID','Datum','PREIS','ANR','RECHNUNG'])
df_grouped = df_new.groupby(
    ['ID']
).agg({'PREIS': 'sum','Not Found':'first'}).reset_index().rename(columns={'PREIS': 'Fakturierte Nettoumsatz'})
df_grouped = df_grouped.merge(df_first,on='ID',how='left')
df_grouped = df_grouped[['ID', 'AWIN ID', 'Publisher_ID', 'Sale_Betrag', 'Provision', 'Datum',
       'Site_Name', 'URL', 'Ablehnungs_Grund', 'Referrer', 'Publisher_URL',
       'Transaktions_Teile', 'Land_nach_IP', 'Benutzerdef_Parameter',
       'Produkte', 'getrackte_Teile', 'Netzwerkgebühr','Fakturierte Nettoumsatz','Not Found',"LAND"]]



In [ ]:
df_grouped

Finally, we compute the difference between our nettoumsatz and what is given by Morefire as Sales_Betrag and leave it in the Difference column.
When everything is computed. we save the data in an excel sheet and there do some minor formatting like formatting the data as a table and then using formatting cell to add Euro sign to columns with Euro values and so on.

In [ ]:

df_grouped['Fakturierte Nettoumsatz'] = df_grouped['Fakturierte Nettoumsatz'].fillna(0)
# df_grouped.loc[df_grouped['Fakturierte Nettoumsatz']<0, 'Fakturierte Nettoumsatz'] = 0 
    
# 3. Create 'Difference' column
def calculate_difference(row):
    sale_betrag = row['Sale_Betrag']
    fakturierte_nettoumsatz = row['Fakturierte Nettoumsatz']
    difference = sale_betrag - fakturierte_nettoumsatz
    return '' if round(difference, 0) == 0 else f"{round(difference, 2):.2f}"

df_grouped['Difference'] = df_grouped.apply(calculate_difference, axis=1)


In [ ]:
df_grouped

In [ ]:
# df_grouped.to_excel('morefire_list.xlsx', index=False)

with pd.ExcelWriter('MoreFire_Export.xlsx', engine='openpyxl') as writer:
    for land in ['DE','AT','FR','CH']:
        df_final = df_grouped[df_grouped['LAND'] == land].copy()
        df_final.drop(columns=['LAND'],inplace=True)
        df_final.to_excel(writer,sheet_name=land,index=False)